In [1]:
import os
import pandas as pd 
import math
import os
import numpy as np

If bed_start and bed_end are -1 that means that the alignment does not map to the interval. Another way you can come to this conclusion is by calculating the fraction of the read that overlaps and setting some cutoffs for what you consider VNTR, junction and Non-VNTR. 

In [16]:
# SETTING case specific variables 
case = 1
seq_len = 10000
vntr_len = 65
coverage = 30

# SETTING project specific variables 
proj_dir = '/frazer01/home/joreyna/repos/CSE-280a/Project/'
sample = 'sequence_{}_case_{}'.format(seq_len, case)
coverage_text = 'coverage_{}'.format(coverage)
out_dir = os.path.join(proj_dir, 'output/pipeline/sample/',  sample + '/', coverage_text + '/')

# WOA file
wao_fn = os.path.join(out_dir, sample + '.wao')   

# PARSE the bedtools intersect -wao result 
data = pd.read_table(wao_fn, header=None)
data.columns = ['Map_Template', 'Map_Start', 'Map_End', 'Read_Name', \
                '4', 'Read_Orientation', '6', '7', '8', '9', \
                'Map_BPs_Aligned', '11', 'Bed_Template', 'Bed_Start', 'Bed_End', 'Overlap']
data = data[['Map_Template', 'Map_Start', 'Map_End', 'Read_Name', \
                'Read_Orientation', 'Map_BPs_Aligned', 'Bed_Template', \
                 'Bed_Start', 'Bed_End', 'Overlap']]
data['Map_Start'] = pd.to_numeric(data['Map_Start'])
data['Map_End'] = pd.to_numeric(data['Map_End'])
data['Map_BPs_Aligned'] = [int(x.replace(',', '')) for x in data['Map_BPs_Aligned']]
data['Bed_Start'] = pd.to_numeric(data['Bed_Start'])
data['Bed_End'] = pd.to_numeric(data['Bed_End'])
data['Overlap'] = pd.to_numeric(data['Overlap'])

# FILTER out reads whose mapping length is not 100 bp's 
data = data[(data['Map_BPs_Aligned'] <= 150) & (data['Map_BPs_Aligned'] >= 50)] 

# ADD a copy number column from the template sequence 
data['Map_Copy_Number'] = data['Map_Template'].str.extract('seq([0-9]*)', expand=True)
data = data[['Map_Template',
 'Map_Copy_Number',
 'Map_Start',
 'Map_End',
 'Read_Name',
 'Read_Orientation',
 'Map_BPs_Aligned',
 'Bed_Template',
 'Bed_Start',
 'Bed_End',
 'Overlap',
 ]]
data = data[data['Overlap'] > 0]

In [17]:
data.head(10)

,Map_Template,Map_Copy_Number,Map_Start,Map_End,Read_Name,Read_Orientation,Map_BPs_Aligned,Bed_Template,Bed_Start,Bed_End,Overlap
2035,seq1,1,4983,5133,seq1-1955,-,150,seq1,5000,5065,65
2131,seq3,3,5170,5320,seq1-1859,+,150,seq3,5000,5195,25
2133,seq1,1,4984,5134,seq1-1857,+,150,seq1,5000,5065,65
2180,seq1,1,4875,5025,seq1-1810,+,150,seq1,5000,5065,25
2183,seq3,3,4891,5041,seq1-1807,-,150,seq3,5000,5195,41
2195,seq1,1,4992,5142,seq1-1795,-,150,seq1,5000,5065,65
2368,seq3,3,4890,5040,seq1-1622,-,150,seq3,5000,5195,40
2390,seq1,1,5004,5154,seq1-1600,-,150,seq1,5000,5065,61
2437,seq1,1,4979,5129,seq1-1553,-,150,seq1,5000,5065,65
2466,seq3,3,4861,5011,seq1-1524,+,150,seq3,5000,5195,11


In [18]:
# def determine_aln_type(map_start, map_end, bed_start, bed_end):
    
#     """
#     Determine the alignment type based on the mapping locations and the bed file locations.
#     """
    
#     if (map_start < bed_start and map_end > bed_end):
#         return 'mix'
#     elif (map_start > bed_start) and (map_end < bed_end):
#         return 'in' 
    
#     elif (map_start < bed_start) and (map_end < bed_start): # left side
#         return 'out'
    
#     elif (map_start > bed_end) and (map_end > bed_end): # right side 
#         return 'out'

#     elif (map_start < bed_start) and (map_end > bed_start): # left side
#         return 'mix'
    
#     elif (map_start < bed_end) and (map_end > bed_end): # right side 
#         return 'mix'

    
#     else:
#         raise Exception('Missing some case. Debug.')

In [19]:
# mapping_status = []
# fractional_vntr = []
# fractional_non_vntr = []
# for index, sr in data.iterrows():
#     read_type = determine_aln_type(sr['Map_Start'], sr['Bed_Start'], sr['Map_End'], sr['Map_Start'])
#     mapping_status.append(read_type)
    

In [20]:
# data['Mapping_Status'] = mapping_status

In [21]:
#data.to_excel('/frazer01/home/joreyna/shared_drive/CSE-280a/data.xlsx')

In [33]:
VNTR_read_count = []
for index, sr in data.iterrows():
    if sr['Overlap'] == sr['Bed_End'] - sr['Bed_Start']:
        VNTR_read_count.append(1)
    else:
        VNTR_read_count.append(float(sr['Overlap']) / (sr['Bed_End'] - sr['Bed_Start']))

In [38]:
data['VNTR_Read_Count'] =  VNTR_read_count
data['Non-VNTR_Read_Count'] = [1 - x for x in VNTR_read_count]

In [39]:
data.head(10)

,Map_Template,Map_Copy_Number,Map_Start,Map_End,Read_Name,Read_Orientation,Map_BPs_Aligned,Bed_Template,Bed_Start,Bed_End,Overlap,VNTR_Read_Count,Non-VNTR_Read_Count
2035,seq1,1,4983,5133,seq1-1955,-,150,seq1,5000,5065,65,1.000000,0.000000
2131,seq3,3,5170,5320,seq1-1859,+,150,seq3,5000,5195,25,0.128205,0.871795
2133,seq1,1,4984,5134,seq1-1857,+,150,seq1,5000,5065,65,1.000000,0.000000
2180,seq1,1,4875,5025,seq1-1810,+,150,seq1,5000,5065,25,0.384615,0.615385
2183,seq3,3,4891,5041,seq1-1807,-,150,seq3,5000,5195,41,0.210256,0.789744
2195,seq1,1,4992,5142,seq1-1795,-,150,seq1,5000,5065,65,1.000000,0.000000
2368,seq3,3,4890,5040,seq1-1622,-,150,seq3,5000,5195,40,0.205128,0.794872
2390,seq1,1,5004,5154,seq1-1600,-,150,seq1,5000,5065,61,0.938462,0.061538
2437,seq1,1,4979,5129,seq1-1553,-,150,seq1,5000,5065,65,1.000000,0.000000
2466,seq3,3,4861,5011,seq1-1524,+,150,seq3,5000,5195,11,0.056410,0.943590


In [40]:
data['Map_Template'].unique()

array(['seq1', 'seq3', 'seq2'], dtype=object)

In [41]:
data['VNTR_Read_Count'].sum() / data['Non-VNTR_Read_Count'].sum()

1.3391744090321063